In [1]:
import BioSimSpace as bss
import sys
from BioSimSpace import _Exceptions
import os
from tqdm import tqdm

/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [2]:
import Sire

In [3]:
bss.FreeEnergy.engines()

['Somd', 'Gromacs']

In [2]:
bss.IO.fileFormats()

['Gro87', 'GroTop', 'MOL2', 'PDB', 'PRM7', 'PSF', 'RST', 'RST7', 'SDF']

In [6]:
index = 0
ligand_stream = open("../ligands.dat", "r")
ligand_lines = ligand_stream.readlines()
ligand_name = ligand_lines[index].rstrip()
ligand_solvated = bss.IO.readMolecules([f"../inputs/ligands/{ligand_name}_ligand_solvated.prm7",
                                        f"../inputs/ligands/{ligand_name}_ligand_solvated.rst7"])
system_solvated = bss.IO.readMolecules([f"../inputs/ligands/{ligand_name}_system_solvated.prm7",
                                        f"../inputs/ligands/{ligand_name}_system_solvated.rst7"])


In [52]:
def run_process(system, md_protocol):
    """
    @param system: solvated system (bss object)
    @param md_protocol: bss protocolbss protocol
    @return the process.
    """
    process = bss.Process.Gromacs(system, md_protocol)
    # process.setArgs({"-nt": 1, "-nb": "gpu", "-gpu_id": 0})
    process.setArg("-nt", 1)
    process.setArg("-v", "yes")
    # process.setArg("-nb", "gpu")
    # process.setArg("-gpu_id", 0)
    process.start()
    process.wait()
    print(f"\n {process}")
    print(f"\n Runtime: {process.runTime()}")
    if process.isError():
        print(process.stdout())
        print(process.stderr())
        raise _Exceptions.ThirdPartyError("The process exited with an error!")
    return process



In [53]:
minimisation_steps = 10
runtime_short_nvt = 5  # ps
runtime_nvt = 50  # ps
runtime_npt = 200  # ps

In [56]:
print("---------------------------")
print("Working on solvated ligand.")
print(f"\nMinimising in {minimisation_steps} steps.")
protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
minimised_process = run_process(ligand_solvated, protocol)
minimised = minimised_process.getSystem()
print("Finished minimisation.")

---------------------------
Working on solvated ligand.
Minimising in 10 steps.

 <BioSimSpace.Process.Gromacs: system=<BioSimSpace.System: nMolecules=666>, protocol=BioSimSpace.Protocol.Minimisation(steps=10), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmphhsycpwf', seed=None>

 Runtime: 0.0034 mins
Finished minimisation.


In [57]:
print(f"\nNVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms.")
protocol = bss.Protocol.Equilibration(
                                    runtime=runtime_short_nvt*bss.Units.Time.picosecond,
                                    temperature_start=0*bss.Units.Temperature.kelvin,
                                    temperature_end=300*bss.Units.Temperature.kelvin,
                                    restraint="all"
                                )
restrained_nvt_process = run_process(minimised, protocol)
restrained_nvt = restrained_nvt_process.getSystem()

NVT equilibration for 5 ps while restraining all non-solvent atoms.

 <BioSimSpace.Process.Gromacs: system=<BioSimSpace.System: nMolecules=666>, protocol=BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=5.0000 ps, temperature_start=0.0100 K, temperature_end=300.0000 K, pressure=None, report_interval=100, restart_interval=500, restraint='all', force_constant=10.00), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmp5jak2_gb', seed=None>

 Runtime: 0.0650 mins


In [58]:
print(f"\nNVT equilibration for {runtime_nvt} ps without restraints.")
protocol = bss.Protocol.Equilibration(
                                runtime=runtime_nvt*bss.Units.Time.picosecond,
                                temperature=300*bss.Units.Temperature.kelvin,
                                )

nvt_process = run_process(restrained_nvt, protocol)
nvt = nvt_process.getSystem()

NVT equilibration for 50 ps without restraints.

 <BioSimSpace.Process.Gromacs: system=<BioSimSpace.System: nMolecules=666>, protocol=BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=50.0000 ps, temperature_start=300.0000 K, temperature_end=300.0000 K, pressure=None, report_interval=100, restart_interval=500, restraint=None, force_constant=10.00), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmp5rfvdrzj', seed=None>

 Runtime: 0.6281 mins


In [59]:
print(f"\nNPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms.")
protocol = bss.Protocol.Equilibration(
                                runtime=runtime_npt*bss.Units.Time.picosecond,
                                pressure=1*bss.Units.Pressure.atm,
                                temperature=300*bss.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
restrained_npt_process = run_process(nvt, protocol)
restrained_npt = restrained_npt_process.getSystem()


NPT equilibration for 200 ps while restraining non-solvent heavy atoms.

 <BioSimSpace.Process.Gromacs: system=<BioSimSpace.System: nMolecules=666>, protocol=BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=200.0000 ps, temperature_start=300.0000 K, temperature_end=300.0000 K, pressure=1.0000 atm, report_interval=100, restart_interval=500, restraint='heavy', force_constant=10.00), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmpw0f9157h', seed=None>

 Runtime: 2.5507 mins


In [61]:
index = 5
os.system(f"touch test_{index}.txt")

0

In [ ]:

output_files = minimised_process.workDir()
os.system(f"cp {output_files}/* ../equilibration_files/equilibration_{index}/")
